# 📊 **New York Times Comments Dataset Analysis**
This notebook analyzes the New York Times Comments dataset available on Kaggle.
We will extract metadata, check for missing values, and summarize the structure of the dataset before proceeding with text analysis.

## **📌 Step 1: Setup the Environment**
We start by importing the necessary libraries and listing all available files in the dataset.

In [1]:
import os
import pandas as pd

# Set path to dataset (Kaggle users should adjust as needed)
dataset_path = "/kaggle/input/nyt-comments"

# List all files in the dataset
files = os.listdir(dataset_path)
print("Files in dataset:\n", files)

Files in dataset:
 ['CommentsFeb2018.csv', 'ArticlesFeb2017.csv', 'CommentsApril2018.csv', 'ArticlesJan2017.csv', 'ArticlesMay2017.csv', 'CommentsJan2017.csv', 'CommentsMarch2017.csv', 'CommentsMay2017.csv', 'CommentsMarch2018.csv', 'CommentsApril2017.csv', 'ArticlesMarch2017.csv', 'ArticlesApril2017.csv', 'CommentsFeb2017.csv', 'ArticlesJan2018.csv', 'ArticlesFeb2018.csv', 'ArticlesMarch2018.csv', 'CommentsJan2018.csv', 'ArticlesApril2018.csv']


## **📌 Step 2: Load & Inspect Data**
Let's load files (e.g., `ArticlesJan2017.csv` and `CommentsApril2017.csv`) to inspect its structure.

In [2]:
# Load an example file to inspect its structure
sample_file = "ArticlesJan2017.csv"  # You can change this to any file in the dataset
df = pd.read_csv(os.path.join(dataset_path, sample_file))

# Display first few rows
df.head()

,articleID,abstract,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL,articleWordCount
0,58691a5795d0e039260788b9,NaN,By JENNIFER STEINHAUER,article,G.O.P. Leadership Poised to Topple Obama’s Pi...,"['United States Politics and Government', 'Law...",1,National,1,2017-01-01 15:03:38,Politics,The most powerful and ambitious Republican-led...,The New York Times,News,https://www.nytimes.com/2017/01/01/us/politics...,1324
1,586967bf95d0e03926078915,NaN,By MARK LANDLER,article,Fractured World Tested the Hope of a Young Pre...,"['Obama, Barack', 'Afghanistan', 'United State...",1,Foreign,1,2017-01-01 20:34:00,Asia Pacific,A strategy that went from a “good war” to the ...,The New York Times,News,https://www.nytimes.com/2017/01/01/world/asia/...,2836
2,58698a1095d0e0392607894a,NaN,By CAITLIN LOVINGER,article,Little Troublemakers,"['Crossword Puzzles', 'Boxing Day', 'Holidays ...",1,Games,0,2017-01-01 23:00:24,Unknown,Chuck Deodene puts us in a bubbly mood.,The New York Times,News,https://www.nytimes.com/2017/01/01/crosswords/...,445
3,5869911a95d0e0392607894e,NaN,By JOCHEN BITTNER,article,"Angela Merkel, Russia’s Next Target","['Cyberwarfare and Defense', 'Presidential Ele...",1,OpEd,15,2017-01-01 23:30:27,Unknown,"With a friend entering the White House, Vladim...",The New York Times,Op-Ed,https://www.nytimes.com/2017/01/01/opinion/ang...,864
4,5869a61795d0e03926078962,NaN,By JIAYIN SHEN,article,Boots for a Stranger on a Bus,"['Shoes and Boots', 'Buses', 'New York City']",0,Metro,12,2017-01-02 01:00:02,Unknown,Witnessing an act of generosity on a rainy day.,The New York Times,Brief,https://www.nytimes.com/2017/01/01/nyregion/me...,309


In [9]:
# Load an example file to inspect its structure
sample_file_2 = "CommentsApril2017.csv"  # You can change this to any file in the dataset
df_2 = pd.read_csv(os.path.join(dataset_path, sample_file_2))

# Display first few rows
df_2.head()

<ipython-input-9-5a2998bf1dac>:3: DtypeWarning: Columns (25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2 = pd.read_csv(os.path.join(dataset_path, sample_file_2))


,approveDate,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,editorsSelection,parentID,...,userLocation,userTitle,userURL,inReplyTo,articleID,sectionName,newDesk,articleWordCount,printPage,typeOfMaterial
0,1491245186,This project makes me happy to be a 30+ year T...,22022598.0,22022598,<br/>,comment,1.491237e+09,1,False,0.0,...,"Riverside, CA",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
1,1491188619,Stunning photos and reportage. Infuriating tha...,22017350.0,22017350,NaN,comment,1.491180e+09,1,False,0.0,...,<br/>,NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
2,1491188617,Brilliant work from conception to execution. I...,22017334.0,22017334,<br/>,comment,1.491179e+09,1,False,0.0,...,Raleigh NC,NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
3,1491167820,NYT reporters should provide a contributor's l...,22015913.0,22015913,<br/>,comment,1.491150e+09,1,False,0.0,...,"Missouri, USA",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
4,1491167815,Could only have been done in print. Stunning.,22015466.0,22015466,<br/>,comment,1.491147e+09,1,False,0.0,...,"Tucson, Arizona",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News


## **📌 Step 3: Extract Metadata**
Now, we extract key metadata, such as column names, data types, and missing values.

In [4]:
# Display dataset information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850 entries, 0 to 849
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   articleID         850 non-null    object
 1   abstract          40 non-null     object
 2   byline            850 non-null    object
 3   documentType      850 non-null    object
 4   headline          850 non-null    object
 5   keywords          850 non-null    object
 6   multimedia        850 non-null    int64 
 7   newDesk           850 non-null    object
 8   printPage         850 non-null    int64 
 9   pubDate           850 non-null    object
 10  sectionName       850 non-null    object
 11  snippet           850 non-null    object
 12  source            850 non-null    object
 13  typeOfMaterial    850 non-null    object
 14  webURL            850 non-null    object
 15  articleWordCount  850 non-null    int64 
dtypes: int64(3), object(13)
memory usage: 106.4+ KB


In [10]:
# Display dataset information
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243832 entries, 0 to 243831
Data columns (total 34 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   approveDate            243832 non-null  int64  
 1   commentBody            243832 non-null  object 
 2   commentID              243832 non-null  float64
 3   commentSequence        243832 non-null  int64  
 4   commentTitle           228498 non-null  object 
 5   commentType            243832 non-null  object 
 6   createDate             243832 non-null  float64
 7   depth                  243832 non-null  int64  
 8   editorsSelection       243832 non-null  bool   
 9   parentID               243832 non-null  float64
 10  parentUserDisplayName  70526 non-null   object 
 11  permID                 243832 non-null  object 
 12  picURL                 243832 non-null  object 
 13  recommendations        243832 non-null  float64
 14  recommendedFlag        0 non-null   

## **📌 Step 4: Check for Missing Values**
Checking for missing values in each column.

In [5]:
# Check for missing values
missing_values = df.isnull().sum()
missing_values[missing_values > 0]

abstract    810
dtype: int64

In [11]:
# Check for missing values
missing_values_2 = df_2.isnull().sum()
missing_values_2[missing_values_2 > 0]

commentTitle              15334
parentUserDisplayName    173306
recommendedFlag          243832
reportAbuseFlag          243832
userDisplayName              77
userLocation                 62
userTitle                243791
userURL                  243827
dtype: int64

## **📌 Step 5: Summary Statistics**
Generate a summary of numeric and categorical columns.

In [6]:
# Display summary statistics
df.describe(include="all").transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
articleID,850,850,58691a5795d0e039260788b9,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abstract,40,40,"After losing three limbs in Afghanistan, a Mar...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
byline,850,434,By THE EDITORIAL BOARD,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
documentType,850,2,article,810,NaN,NaN,NaN,NaN,NaN,NaN,NaN
headline,850,774,Unknown,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN
keywords,850,717,[],73,NaN,NaN,NaN,NaN,NaN,NaN,NaN
multimedia,850.0,NaN,NaN,NaN,0.927059,0.260193,0.0,1.0,1.0,1.0,1.0
newDesk,850,28,OpEd,175,NaN,NaN,NaN,NaN,NaN,NaN,NaN
printPage,850.0,NaN,NaN,NaN,7.077647,10.100022,0.0,0.0,1.0,12.0,66.0
pubDate,850,786,2017-02-02 08:21:23,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Display summary statistics
df_2.describe(include="all").transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
approveDate,243832.0,NaN,NaN,NaN,1492504461.72283,1330667.064463,1491008047.0,1491783853.25,1492437034.5,1493119511.0,1524346252.0
commentBody,243832,243169,Well said.,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
commentID,243832.0,NaN,NaN,NaN,22188608.516839,185515.004384,21999548.0,22092910.75,22176681.5,22260467.5,26824246.0
commentSequence,243832.0,NaN,NaN,NaN,22188608.516839,185515.004384,21999548.0,22092910.75,22176681.5,22260467.5,26824246.0
commentTitle,228498,1,<br/>,228498,NaN,NaN,NaN,NaN,NaN,NaN,NaN
commentType,243832,3,comment,173277,NaN,NaN,NaN,NaN,NaN,NaN,NaN
createDate,243832.0,NaN,NaN,NaN,1492495336.431272,1328516.276157,1491006872.0,1491768493.75,1492428038.0,1493086582.25,1524345694.0
depth,243832.0,NaN,NaN,NaN,1.289425,0.453641,1.0,1.0,1.0,2.0,3.0
editorsSelection,243832,2,False,238159,NaN,NaN,NaN,NaN,NaN,NaN,NaN
parentID,243832.0,NaN,NaN,NaN,6416284.403421,10055291.992253,0.0,0.0,0.0,22051084.75,26426201.0


## **📌 Step 6: Check for Unique Identifiers**
Find columns that can be used as unique identifiers.

In [7]:
# Check if any column can be used as a unique identifier
unique_counts = df.nunique()
unique_counts

articleID           850
abstract             40
byline              434
documentType          2
headline            774
keywords            717
multimedia            2
newDesk              28
printPage            43
pubDate             786
sectionName          30
snippet             846
source                2
typeOfMaterial       11
webURL              850
articleWordCount    689
dtype: int64

In [13]:
# Check if any column can be used as a unique identifier
unique_counts_2 = df_2.nunique()
unique_counts_2

approveDate              115718
commentBody              243169
commentID                243832
commentSequence          243832
commentTitle                  1
commentType                   3
createDate               228348
depth                         3
editorsSelection              2
parentID                  41494
parentUserDisplayName     15712
permID                   243832
picURL                     4282
recommendations            1232
recommendedFlag               0
replyCount                   79
reportAbuseFlag               0
sharing                       2
status                        1
timespeople                   2
trusted                       2
updateDate               136865
userDisplayName           46510
userID                    62946
userLocation              15890
userTitle                     9
userURL                       1
inReplyTo                 41494
articleID                   886
sectionName                  31
newDesk                      28
articleW

## **📌 Step 7: Automate Metadata Extraction for All Files**
Instead of manually inspecting each file, we automate metadata extraction for all files.

In [8]:
# Iterate over all files and extract metadata
metadata_summary = []

for file in files:
    file_path = os.path.join(dataset_path, file)
    df = pd.read_csv(file_path)

    metadata_summary.append({
        "File Name": file,
        "Rows": df.shape[0],
        "Columns": df.shape[1],
        "Missing Values": df.isnull().sum().sum(),
        "Duplicate Rows": df.duplicated().sum(),
        "Unique Columns": df.nunique().to_dict(),
    })

# Convert metadata summary to DataFrame for better readability
metadata_df = pd.DataFrame(metadata_summary)
metadata_df

<ipython-input-8-a0b18595323b>:6: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-8-a0b18595323b>:6: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-8-a0b18595323b>:6: DtypeWarning: Columns (14,15,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-8-a0b18595323b>:6: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-8-a0b18595323b>:6: DtypeWarning: Columns (25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-8-a0b18595323b>:6: DtypeWarning: Columns (14,15,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(

,File Name,Rows,Columns,Missing Values,Duplicate Rows,Unique Columns
0,CommentsFeb2018.csv,215282,34,1018593,0,"{'approveDate': 158054, 'articleID': 1155, 'ar..."
1,ArticlesFeb2017.csv,885,16,855,0,"{'articleID': 885, 'abstract': 30, 'byline': 4..."
2,CommentsApril2018.csv,264924,34,1240919,0,"{'approveDate': 196777, 'articleID': 1351, 'ar..."
3,ArticlesJan2017.csv,850,16,810,0,"{'articleID': 850, 'abstract': 40, 'byline': 4..."
4,ArticlesMay2017.csv,996,16,963,0,"{'abstract': 33, 'articleID': 996, 'articleWor..."
5,CommentsJan2017.csv,231449,34,1114483,0,"{'approveDate': 106710, 'articleID': 850, 'art..."
6,CommentsMarch2017.csv,260967,34,1249140,0,"{'approveDate': 115903, 'articleID': 949, 'art..."
7,CommentsMay2017.csv,276389,34,1322148,0,"{'approveDate': 160236, 'commentBody': 275493,..."
8,CommentsMarch2018.csv,246915,34,1331416,0,"{'approveDate': 187256, 'articleID': 1385, 'ar..."
9,CommentsApril2017.csv,243832,34,1164061,0,"{'approveDate': 115718, 'commentBody': 243169,..."


## **🔍 Conclusion**
This notebook provides insights into the dataset structure, missing values, and metadata, making it ready for further text processing and LSTM-based text generation analysis.

# 📊 **LSTM-Based Text Generation on NYT Comments Dataset**
This notebook trains an LSTM model using the **New York Times Comments dataset** to generate human-like text. The notebook follows a structured process: merging datasets, preprocessing text, tokenization, training an LSTM model, and saving progress to prevent data loss in case of session shutdown.

## **📌 Step 1: Load & Merge All Comment Datasets**
We combine all comments into a single dataset for better model generalization.

In [2]:
import os
import pandas as pd

# Path to dataset directory
dataset_path = "/kaggle/input/nyt-comments"

# List all comment files
comment_files = [file for file in os.listdir(dataset_path) if file.startswith("Comments")]

# Initialize empty list to store DataFrames
df_list = []

# Load and merge all comment files
for file in comment_files:
    file_path = os.path.join(dataset_path, file)
    df = pd.read_csv(file_path, usecols=["commentBody"])
    df_list.append(df)

# Combine all comments into one DataFrame
df_combined = pd.concat(df_list, ignore_index=True)

# Display dataset shape before cleaning
print("Total Comments (Before Cleaning):", df_combined.shape[0])
df_combined.head()

Total Comments (Before Cleaning): 2176364


,commentBody
0,The snake-filled heads comment made me think o...
1,She-devil reporting for duty!
2,XX is the new mark of the devil.
3,"""Courtland Sykes"" should be writing for The On..."
4,"I happen to descend for a few of them, because..."


## **📌 Step 2: Preprocessing the Text**
We clean the text by converting to lowercase, removing special characters, and tokenizing words into sequences.

In [4]:
import re
import json
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tqdm.pandas()

# Function to clean text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

# Apply text cleaning with progress bar
print("\n🔄 Cleaning text data...")
df_combined["commentBody"] = df_combined["commentBody"].astype(str).progress_apply(clean_text)

# ✅ Save AFTER cleaning
df_combined.to_csv("nyt_comments_cleaned.csv", index=False)
print("\n✅ Cleaned dataset saved as 'nyt_comments_cleaned.csv'")

# Display sample cleaned text
print("\n📌 Sample cleaned text:\n", df_combined["commentBody"].head(5))


🔄 Cleaning text data...


100%|██████████| 2176364/2176364 [01:48<00:00, 20001.10it/s]



✅ Cleaned dataset saved as 'nyt_comments_cleaned.csv'

📌 Sample cleaned text:
 0    the snakefilled heads comment made me think of...
1                          shedevil reporting for duty
2                      xx is the new mark of the devil
3    courtland sykes should be writing for the onio...
4    i happen to descend for a few of them because ...
Name: commentBody, dtype: object


In [5]:
import pandas as pd

print("\n📂 Loading cleaned dataset 'nyt_comments_cleaned.csv'...")
df_combined = pd.read_csv("nyt_comments_cleaned.csv")

print(f"✅ Loaded dataset with {df_combined.shape[0]} rows.")
print(df_combined.head())  # Preview first few rows



📂 Loading cleaned dataset 'nyt_comments_cleaned.csv'...
✅ Loaded dataset with 2176364 rows.
                                         commentBody
0  the snakefilled heads comment made me think of...
1                        shedevil reporting for duty
2                    xx is the new mark of the devil
3  courtland sykes should be writing for the onio...
4  i happen to descend for a few of them because ...


In [6]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import numpy as np
import json
from tqdm import tqdm

# ✅ Set vocabulary size & sequence length
max_tokens = 50000  # Adjusted to avoid memory overload
sequence_length = 30  # Limit sequence length
batch_size = 10000  # Process comments in batches

print("\n🔄 Initializing TextVectorization layer...")
vectorizer = TextVectorization(max_tokens=max_tokens, output_sequence_length=sequence_length)

# ✅ Convert comments to NumPy array & Ensure all are strings
print("\n🔄 Preparing comments array...")
comments_array = df_combined["commentBody"].astype(str).fillna("").to_numpy()  # Convert NaNs & floats to strings

# ✅ Fit vectorizer to text data
print("\n🔄 Fitting vectorizer on text data...")
vectorizer.adapt(comments_array)

# ✅ Convert text to sequences in batches (Fixes tqdm printing issues)
print("\n🔄 Converting text to tokenized sequences...")
sequences = []

for i in tqdm(range(0, len(comments_array), batch_size), desc="Processing text", ncols=100, dynamic_ncols=True, leave=True):
    batch = comments_array[i : i + batch_size]  # Process in batches
    batch_sequences = vectorizer(batch).numpy()  # Vectorize batch at once
    sequences.extend(batch_sequences)

    # ✅ Force tqdm to only update one line instead of multiple bars
    tqdm.write(f"✅ Processed {min(i+batch_size, len(comments_array))}/{len(comments_array)} comments...")

# ✅ Convert to NumPy array
sequences = np.array(sequences, dtype=np.int32)

# ✅ Verify Vocabulary Size
tqdm.write(f"\n📌 Vocabulary Size (After Limiting): {vectorizer.vocabulary_size()} words")  # Use tqdm.write to keep it in one line

# ✅ Save vectorizer (Required for future model inference)
tqdm.write("\n💾 Saving vectorizer configuration...")
vectorizer_config = {"max_tokens": max_tokens, "sequence_length": sequence_length}
with open("vectorizer_config.json", "w") as f:
    json.dump(vectorizer_config, f)

# ✅ Save tokenized text before sequence creation
print("\n💾 Saving raw tokenized sequences...")
np.save("tokenized_comments.npy", sequences)  # Changed filename

# ✅ Confirm the process is complete
tqdm.write("\n✅ Tokenization complete!")



🔄 Initializing TextVectorization layer...

🔄 Preparing comments array...

🔄 Fitting vectorizer on text data...

🔄 Converting text to tokenized sequences...


Processing text:   0%|          | 1/218 [00:00<00:40,  5.42it/s]

✅ Processed 10000/2176364 comments...


Processing text:   1%|          | 2/218 [00:00<00:34,  6.21it/s]

✅ Processed 20000/2176364 comments...


...

✅ Processed 2160000/2176364 comments...


Processing text: 100%|█████████▉| 217/218 [00:33<00:00,  3.09it/s]

✅ Processed 2170000/2176364 comments...


Processing text: 100%|██████████| 218/218 [00:33<00:00,  6.41it/s]


✅ Processed 2176364/2176364 comments...

📌 Vocabulary Size (After Limiting): 50000 words

💾 Saving vectorizer configuration...

💾 Saving raw tokenized sequences...

✅ Tokenization complete!


In [7]:
import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences

# ✅ Load tokenized text
print("\n📂 Loading tokenized sequences from 'tokenized_comments.npy'...")
sequences = np.load("tokenized_comments.npy", allow_pickle=True)

# ✅ Ensure the shape is correct
print(f"\n📌 Loaded tokenized sequences with shape: {sequences.shape}")

# ✅ Create input sequences using NumPy arrays (Optimized)
sequence_length = 30  # Ensure consistency
max_sequences = sequences.shape[0]  # Avoid inefficient `sum(len(seq)...)`
input_sequences = np.zeros((max_sequences, sequence_length), dtype=np.int32)

print("\n🔄 Creating input sequences...")
index = 0

for seq in tqdm(sequences, desc="Processing sequences"):
    for i in range(1, len(seq)):
        sub_seq = seq[:i+1]

        # ✅ Fix: Trim sequences that exceed `sequence_length`
        if len(sub_seq) > sequence_length:
            sub_seq = sub_seq[-sequence_length:]  # Keep last 30 tokens
        
        # ✅ Prevent overflow in `input_sequences`
        if index >= max_sequences:
            break

        input_sequences[index, -len(sub_seq):] = sub_seq  # Insert at the end
        index += 1

# ✅ Pad sequences with reduced max length
print("\n🔄 Padding input sequences...")
input_sequences = pad_sequences(input_sequences, maxlen=sequence_length, padding="pre")

print(f"\n📌 Padded input shape: {input_sequences.shape}")

# ✅ Extract input (X) and output (y)
X, y = input_sequences[:, :-1], input_sequences[:, -1]

print(f"\n📌 X shape: {X.shape}, y shape: {y.shape}")

# ✅ Convert y to integer labels (Sparse Encoding)
print("\n🔄 Converting y to sparse labels...")
y = np.array(y, dtype=np.int32)  # Uses sparse categorical encoding

# ✅ Save processed training sequences separately
print("\n💾 Saving LSTM training sequences...")
np.save("X.npy", X)
np.save("y.npy", y)

print("\n✅ Tokenized sequences saved successfully!")



📂 Loading tokenized sequences from 'tokenized_comments.npy'...

📌 Loaded tokenized sequences with shape: (2176364, 30)

🔄 Creating input sequences...


Processing sequences: 100%|██████████| 2176364/2176364 [00:05<00:00, 408875.99it/s]



🔄 Padding input sequences...

📌 Padded input shape: (2176364, 30)

📌 X shape: (2176364, 29), y shape: (2176364,)

🔄 Converting y to sparse labels...

💾 Saving LSTM training sequences...

✅ Tokenized sequences saved successfully!


## **📌 Step 3: Building the LSTM Model**
We define an LSTM-based architecture with embedding and dense layers.

In [7]:
import numpy as np

sequence_length = 30  # Ensure consistency with previous settings

# ✅ Load preprocessed training data
print("\n📂 Loading preprocessed sequences from 'X.npy' and 'y.npy'...")
X = np.load("X.npy", allow_pickle=True)
y = np.load("y.npy", allow_pickle=True)

# ✅ Confirm dataset shapes
print(f"✅ Loaded input sequences. X shape: {X.shape}, y shape: {y.shape}")



📂 Loading preprocessed sequences from 'X.npy' and 'y.npy'...
✅ Loaded input sequences. X shape: (2176364, 29), y shape: (2176364,)


In [3]:
import json
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Input, TextVectorization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import mixed_precision

# ✅ Enable Mixed Precision for Faster Training
mixed_precision.set_global_policy("mixed_float16")

# ✅ Load vectorizer configuration
print("\n📂 Loading vectorizer configuration from 'vectorizer_config.json'...")
with open("vectorizer_config.json", "r") as f:
    vectorizer_config = json.load(f)

# ✅ Restore `TextVectorization` layer with saved settings
max_tokens = vectorizer_config["max_tokens"]
sequence_length = vectorizer_config["sequence_length"]

print(f"\n✅ Restoring vectorizer with max_tokens={max_tokens} and sequence_length={sequence_length}...")

vectorizer = TextVectorization(max_tokens=max_tokens, output_sequence_length=sequence_length)

# ✅ Define vocabulary size from `vectorizer`
vocab_size = vectorizer.vocabulary_size()  # Correct way to get vocab size

print(f"✅ Vocabulary Size: {vocab_size}")

# ✅ Define Optimized LSTM Model
model = Sequential([
    Input(shape=(sequence_length - 1,)),  # Explicit input layer
    Embedding(input_dim=vocab_size, output_dim=256),  # Increased embedding size for better word representation
    Bidirectional(LSTM(256, return_sequences=True)),  # BiLSTM for deeper context understanding
    LSTM(256),  # Additional LSTM layer
    Dropout(0.3),  # Higher dropout to prevent overfitting
    Dense(256, activation="relu"),
    Dense(vocab_size, activation="softmax", dtype="float32")  # Ensure output layer remains float32
])

# ✅ Compile model with Adam optimizer and lower learning rate
model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(learning_rate=0.0005), metrics=["accuracy"])

# ✅ Print model summary
model.summary()



📂 Loading vectorizer configuration from 'vectorizer_config.json'...

✅ Restoring vectorizer with max_tokens=50000 and sequence_length=30...
✅ Vocabulary Size: 2


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 29, 256)             │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 29, 512)             │       1,050,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 256)                 │         787,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │             514 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,904,898 (7.27 MB)

 Trainable params: 1,904,898 (7.27 MB)

 Non-trainable params: 0 (0.00 B)

## **📌 Step 4: Training the LSTM Model**
We train the LSTM model with categorical cross-entropy loss.

In [4]:
import tensorflow as tf

# Check if GPU is available
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 1


In [5]:
# Ensure TensorFlow uses GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # Prevents memory overflow issues
        print("\n✅ GPU is enabled and TensorFlow is using it!")
    except RuntimeError as e:
        print(e)
else:
    print("\n❌ No GPU detected, training may be slow!")


Physical devices cannot be modified after being initialized


In [9]:
import time
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# ✅ Ensure `y` is sparse categorical (integer labels)
y = np.array(y, dtype=np.int32)

# ✅ Ensure mixed precision if enabled
if tf.keras.mixed_precision.global_policy().name == "mixed_float16":
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, dtype=tf.float32)
else:
    loss_fn = "sparse_categorical_crossentropy"

# ✅ Compile model with Adam optimizer (learning rate already set in model definition)
model.compile(loss=loss_fn, optimizer="adam", metrics=["accuracy"])

# ✅ Define Callbacks (Save best model & stop early if validation loss stops improving)
callbacks = [
    ModelCheckpoint("nyt_lstm_best_model.keras", save_best_only=True, monitor="val_loss", mode="min", verbose=1),
    EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True, verbose=1)
]

# ✅ Start Timer
start_time = time.time()
print("\n🚀 Starting Model Training...\n")

# ✅ Reduce training dataset size for quick testing (Adjust as needed)
X_train, y_train = X[:1000000], y[:1000000]

# ✅ Train model with verbose logging and callbacks
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=512,
    validation_split=0.2,  # Ensure data is shuffled before training
    shuffle=True,
    verbose=1,
    callbacks=callbacks
)

# ✅ Compute total training time
end_time = time.time()
total_time = end_time - start_time
print(f"\n✅ Training Completed in {total_time:.2f} seconds ({total_time/60:.2f} minutes)")

# ✅ Save Model (Final version)
model.save("nyt_lstm_model.keras")
print("\n💾 Model saved as 'nyt_lstm_model.keras'")

# ✅ Save Training History
with open("training_history.pkl", "wb") as f:
    pickle.dump(history.history, f)
print("\n📊 Training history saved as 'training_history.pkl'")

# ✅ Print Final Training Stats
print("\n📌 Final Training Metrics:")
print(f"   🔹 Final Training Loss: {history.history['loss'][-1]:.4f}")
print(f"   🔹 Final Validation Loss: {history.history['val_loss'][-1]:.4f}")
print(f"   🔹 Final Training Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"   🔹 Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")

print("\n🎯 Training Complete! You can now evaluate the model and generate text.")



🚀 Starting Model Training...

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node TensorScatterUpdate defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.1

## **📌 Step 5: Generate New Comments Using the LSTM**
We use the trained model to predict and generate text from a given seed phrase.

In [ ]:
import numpy as np

def generate_text(seed_text, next_words=50, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=sequence_length-1, padding="pre")

        # Predict next word probabilities
        predicted_probs = model.predict(token_list, verbose=0)[0]
        
        # Apply temperature to control randomness
        predicted_probs = np.log(predicted_probs) / temperature
        exp_preds = np.exp(predicted_probs)
        predicted_probs = exp_preds / np.sum(exp_preds)

        # Sample from probability distribution
        predicted_index = np.random.choice(len(predicted_probs), p=predicted_probs)

        # Convert index to word
        output_word = tokenizer.index_word.get(predicted_index, "")
        seed_text += " " + output_word
    return seed_text

# Example usage
print(generate_text("the government should", next_words=20, temperature=0.7))  # Try different values


## **📌 Final Summary**
1. **Merged all comment datasets** into a single dataset.
2. **Preprocessed and tokenized the text** for input sequences.
3. **Trained an LSTM model** with embeddings and dense layers.
4. **Saved progress at every stage** to prevent data loss.
5. **Generated new comments** based on seed text input.